# 作者信息 

In [1]:
import xlrd   #引入库
wb=xlrd.open_workbook(r"C:\Users\CANDY\Desktop\广告编码1-200.xlsx")

sheet=wb.sheet_by_index(0)  
bvs=sheet.col_values(4)
mids=sheet.col_values(7)
cids=sheet.col_values(3)
publish_frequencys=sheet.col_values(4)
create_totalDays=sheet.col_values(5)
video_nums=sheet.col_values(6)
like_nums=sheet.col_values(7)

In [29]:
from bs4 import BeautifulSoup
from pandas import DataFrame as df
import pandas
import requests
from lxml import etree
import re
import time
import sys
import os
from bilibili_api import video
import json
import math
import time
import datetime

In [ ]:
headers={
     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:56.0) Gecko/20100101 Firefox/56.0',
     'Host': 'api.bilibili.com',
     'Cookie': 'SESSDATA=e17f7302%2C1688455619%2Cfa3a5*11'
   }
for i in range(1,len(mids)):
    try:
        mid=int(mids[i])
        url_1 = "https://api.bilibili.com/x/web-interface/card?mid=%d"%(mid)
        res_1 = requests.get(url_1)
        info_1 = json.loads(res_1.text)
        video_num = info_1["data"]["archive_count"]
        video_nums[i]= video_num
        like_nums[i]= info_1["data"]["like_num"]

        pn = math.ceil(int(video_num)/30)
        url_3 = "https://api.bilibili.com/x/space/arc/search?mid=%d&pn=%d&ps=30&index=1"%(mid, pn)
        res_3 = requests.get(url_3,headers=headers)
        info_3 = json.loads(res_3.text)
        timestamp = info_3['data']['list']['vlist'][-1]['created']
        #发布频率
        timenow = time.strftime("%Y-%m-%d %H:%M:%S")
        timepast = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))
        time1=datetime.datetime.strptime(timenow,"%Y-%m-%d %H:%M:%S")
        time2 = datetime.datetime.strptime(timepast, "%Y-%m-%d %H:%M:%S")
        create_totalDay= (time1-time2).days
        create_totalDays[i]= create_totalDay
        publish_frequencys[i] = ('%.2f' % (create_totalDay/video_num))
    except:
        pass
        

In [5]:
import pandas as pd
df1 = pd.DataFrame()
df1['video_nums']=video_nums
df1['like_nums'] =like_nums
df1['publish_frequencys']=publish_frequencys
df1['create_totalDays'] =create_totalDays
df1.to_excel(r"C:\Users\CANDY\Desktop\testauthor.xls")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  import sys


# 视频信息

In [2]:
import xlrd   #引入库
wb=xlrd.open_workbook(r"C:\Users\CANDY\Desktop\广告编码1-200.xlsx")
sheet_num=wb.nsheets  #获取excel里面的sheet的数量
sheet_names=wb.sheet_names() 
sheet=wb.sheet_by_index(0)  
bvs=sheet.col_values(8)
#mids=sheet.col_values(2)
cids=sheet.col_values(7)
times=sheet.col_values(1)

In [ ]:
for i in range(1,len(bvs)):
    try:
        bvid=bvs[i]
        video_info=video.get_video_info(bvid=bvid)
        times[i]=video_info['duration']-1
    except:
        pass

In [34]:
import pandas as pd
df1 = pd.DataFrame()
df1['times']=times
df1.to_excel(r"C:\Users\CANDY\Desktop\testtimes.xls")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  after removing the cwd from sys.path.


# 情感

In [15]:
from collections import defaultdict
import os
import re
import jieba
import codecs
#jieba分词后去除停用词
def seg_word(sentence):
    seg_list = jieba.cut(sentence)
    seg_result = []
    for i in seg_list:
        seg_result.append(i)
    stopwords = set()
    with open('stopwords.txt','r',encoding='utf-8') as fr:
        for i in fr:
            stopwords.add(i.strip())
    return list(filter(lambda x :x not in stopwords,seg_result))
#找出文本中的情感词、否定词和程度副词
def classify_words(word_list):
    #读取情感词典文件
    sen_file = open(r'C:\Users\CANDY\代码\BosonNLP_sentiment_score.txt','r+',encoding='utf-8')
    #获取词典文件内容
    sen_list = sen_file.readlines()
    #创建情感字典
    sen_dict = defaultdict()
    #读取词典每一行的内容，将其转换成字典对象，key为情感词，value为其对应的权重
    for i in sen_list:
        if len(i.split(' '))==2:
            sen_dict[i.split(' ')[0]] = i.split(' ')[1]
 
    #读取否定词文件
    not_word_file = open(r'C:\Users\CANDY\代码\否定词.txt','r+',encoding='utf-8')
    not_word_list = not_word_file.readlines()
    #读取程度副词文件
    degree_file = open(r'C:\Users\CANDY\代码\程度副词.txt','r+',encoding='utf-8')
    degree_list = degree_file.readlines()
    degree_dict = defaultdict()
    for i in degree_list:
        degree_dict[i.split(',')[0]] = i.split(',')[1]
    sen_word = dict()
    not_word = dict()
    degree_word = dict()
    #分类
    for i in range(len(word_list)):
        word = word_list[i]
        if word in sen_dict.keys() and word not in not_word_list and word not in degree_dict.keys():
            # 找出分词结果中在情感字典中的词
            sen_word[i] = sen_dict[word]
        elif word in not_word_list and word not in degree_dict.keys():
            # 分词结果中在否定词列表中的词
            not_word[i] = -1
        elif word in degree_dict.keys():
            # 分词结果中在程度副词中的词
            degree_word[i]  = degree_dict[word]
 
 
    #关闭打开的文件
    sen_file.close()
    not_word_file.close()
    degree_file.close()
    #返回分类结果
    return sen_word,not_word,degree_word
#计算情感词的分数
def score_sentiment(sen_word,not_word,degree_word,seg_result):
    #权重初始化为1
    W = 1
    score = 0
    #情感词下标初始化
    sentiment_index = -1
    #情感词的位置下标集合
    sentiment_index_list = list(sen_word.keys())
    #遍历分词结果
    for i in range(0,len(seg_result)):
        #如果是情感词
        if i in sen_word.keys():
            #权重*情感词得分
            score += W*float(sen_word[i])
            #情感词下标加一，获取下一个情感词的位置
            sentiment_index += 1
            if sentiment_index < len(sentiment_index_list)-1:
                #判断当前的情感词与下一个情感词之间是否有程度副词或否定词
                for j in range(sentiment_index_list[sentiment_index],sentiment_index_list[sentiment_index+1]):
                    #更新权重，如果有否定词，权重取反
                    if j in not_word.keys():
                        W *= -1
                    elif j in degree_word.keys():
                        W *= float(degree_word[j])
        #定位到下一个情感词
        if sentiment_index < len(sentiment_index_list)-1:
            i = sentiment_index_list[sentiment_index+1]
    return score
 
#计算得分
def sentiment_score(sentence):
    #1.对文档分词
    seg_list = seg_word(sentence)
    #2.将分词结果转换成字典，找出情感词、否定词和程度副词
    sen_word,not_word,degree_word = classify_words(seg_list)
    #3.计算得分
    score = score_sentiment(sen_word,not_word,degree_word,seg_list)
    return score

In [16]:
import xlrd   #引入库
wb=xlrd.open_workbook(r"C:\Users\CANDY\Desktop\广告编码1-200.xlsx")
sheet_num=wb.nsheets  #获取excel里面的sheet的数量
sheet_names=wb.sheet_names() 
sheet=wb.sheet_by_index(1)  
cids=sheet.col_values(8)
ds=sheet.col_values(0)
ts=sheet.col_values(1)
begin1=sheet.col_values(3)
end1=sheet.col_values(4)
begin2=sheet.col_values(5)
end2=sheet.col_values(6)

In [17]:
# 导入模块
from bilibili_api import video

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
for i in range(1,len(cids)):
    cid=int(cids[i])
    url = 'https://comment.bilibili.com/%d.xml'%(cid)
    html = requests.get(url).content
    html_data = str(html, 'utf-8')
    bs4 = BeautifulSoup(html_data, 'lxml')
    danmu_list= bs4.find_all('d')
    danmuscores=[]
    textscores=[]
    if end1[i]==end2[i]:
        ds[i]=0
        for d in danmu_list:
            postscore=sentiment_score(content)    
            textscores.append(postscore)
        ts[i]=sum(textscores)/len(textscores)
    else:
        for d in danmu_list:
            content=d.text
            total_seconds=int(d['p'].split('.')[0])
            postscore=sentiment_score(content)
            if (begin1[i]<=total_seconds<=end1[i]+2 or begin2[i]-1<total_seconds<=end2[i]+2):
                danmuscores.append(postscore)
            else:
                textscores.append(postscore)       
        try:
            ds[i]=sum(danmuscores)/len(danmuscores)
        except:
            ds[i]=0
        try:
            ts[i]=sum(textscores)/len(textscores)
        except:
            ts[i]=ds[i]   

In [19]:
ds,ts

(['',
  1.9198217450559998,
  0.6190793026571083,
  0.8743970394169,
  0.12183210608807499,
  -0.5193115100048188,
  0.6345380709238674,
  1.4057024334129804,
  0.9459278433048,
  0.9015153756547861,
  0.3937877264087594,
  0.6637750767472964,
  1.0223204493877671,
  0.869812389809175,
  -0.4363225659081072,
  0.6417631823341381,
  -0.2977491653472641,
  0.594889265509279,
  0.7248088583122857,
  0.8442198681249999,
  -0.15121650361465758,
  4.222539593556,
  -0.28348608749485216,
  1.641446126275568,
  0.6386446320259214,
  -0.3789476077861763,
  1.334616987912575,
  0.5860691340067344,
  1.058076626915275,
  1.243260811313564,
  0.6520372645433917,
  0.3438045556366022,
  0.9408939207106067,
  -0.700769972923662,
  0.9663499072952428,
  -0.4698783344111249,
  1.0332997135527628,
  0.9129573020999999,
  0.9278318156059415,
  1.6186031781333756,
  -0.18368977927003766,
  3.533524347853827,
  0.14615733290477778,
  0.8731085545833626,
  0.8530231577253976,
  0.6189504507346134,
  0.7688

In [20]:
import pandas as pd
df1 = pd.DataFrame()
df1['ds']=ds
df1['ts']=ts
df1.to_excel(r"C:\Users\CANDY\Desktop\testqingan.xls")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """
